In [4]:
import pandas as pd
import psycopg2  # noqa: F401 (import ensures the dialect is available)
from sqlalchemy import create_engine, text
import warnings

warnings.filterwarnings("ignore")

# ========= 1) Credentials / connection (fill in user_name & password) =========
user_name = 'carole_audrey_same'
password  = '8vQa8KQb4QxKo6'
host = 'localhost'
port = '5433'
database = 'layereddb'
schema = 'berlin_source_data'

engine = create_engine(f'postgresql+psycopg2://{user_name}:{password}@{host}:{port}/{database}')

# ========= 2) Load cleaned CSV into a pandas DataFrame =========
# This is the file we generated earlier (district fixed to 12 Bezirke, postal_code 0 -> "Unknown", etc.)
csv_path = "kindergartens_berlin_final_ready (3).csv"
kg = pd.read_csv("kindergartens_berlin_final_ready (3).csv")

In [6]:
kg

,kindergarten_id,district_id,name,address,postal_code,latitude,longitude,neighborhood,district,operator,capacity,source,created_at,updated_at
0,1,11012012.0,Bunte Klänge Kindergarten,"Graf-Haeseler-Straße 23, 13403 Berlin",13403,52.569960,13.313863,Reinickendorf,Reinickendorf,Unknown,Unknown,openstreetmap,18/08/2025 11:28,18/08/2025 11:28
1,2,11002002.0,Nestwärme Kita,Unknown,Unknown,52.501536,13.433399,Kreuzberg,Friedrichshain-Kreuzberg,Unknown,Unknown,openstreetmap,18/08/2025 11:28,18/08/2025 11:28
2,3,11007007.0,Kita Fehlerstraße 2,"Fehlerstraße 2, 12161 Berlin",12161,52.476741,13.325825,Friedenau,Tempelhof-Schöneberg,Unknown,Unknown,openstreetmap,18/08/2025 11:28,18/08/2025 11:28
3,4,11007007.0,Ganztagsbetreuung der Fläming-Grundschule,"Rheinstraße 54, 12161 Berlin",12161,52.468522,13.332504,Friedenau,Tempelhof-Schöneberg,Nachbarschaftsheim Schöneberg e.V.,Unknown,openstreetmap,18/08/2025 11:28,18/08/2025 11:28
4,5,11001001.0,kidsweb.de Kindertagesbetreuung,Unknown,Unknown,52.539884,13.349624,Wedding,Mitte,Unknown,Unknown,openstreetmap,18/08/2025 11:28,18/08/2025 11:28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2293,2294,11011011.0,Evangelischer Kindergarten Morgensonne,"Lehndorffstraße 11-15, 10318 Berlin",10318,NaN,NaN,NaN,Lichtenberg,Evangelische Paul-Gerhardt-Kirchengemeinde Lic...,61,openstreetmap,18/08/2025 11:28,18/08/2025 11:28
2294,2295,11001001.0,Kita auf Augenhöhe e.V.,"Lehrter Straße 54, 10557 Berlin",10557,NaN,NaN,NaN,Mitte,Kita auf Augenhöhe e.V.,23,openstreetmap,18/08/2025 11:28,18/08/2025 11:28
2295,2296,NaN,Kita St. Konrad,"Rubensstraße 74, 12157 Berlin",12157,NaN,NaN,NaN,NaN,Hedi Kitas Erzbistum Berlin,Unknown,openstreetmap,18/08/2025 11:28,18/08/2025 11:28
2296,2297,11012012.0,Kita Kinderland,"Friedrich-Wilhelm-Straße 76, 13409 Berlin",13409,NaN,NaN,NaN,Reinickendorf,LebensWelt gGmbH,Unknown,openstreetmap,18/08/2025 11:28,18/08/2025 11:28


In [10]:

# ========= 3) Create schema + tables (PK/FK) =========
create_objects_sql = f"""





-- 3.4 Kindergartens table (final)
CREATE TABLE IF NOT EXISTS berlin_source_data.kindergartens (
    kindergarten_id INTEGER PRIMARY KEY,
    name            VARCHAR(255),
    address         TEXT,
    postal_code     VARCHAR(20),
    latitude        DOUBLE PRECISION,
    longitude       DOUBLE PRECISION,
    district_id     VARCHAR(20) NOT NULL,
    operator        VARCHAR(255),
    capacity        INTEGER,
    source          VARCHAR(255),
    created_at      TIMESTAMP NULL,
    updated_at      TIMESTAMP NULL,
    CONSTRAINT fk_kg_district
        FOREIGN KEY (district_id) REFERENCES berlin_source_data.districts(district_id)
            ON UPDATE CASCADE ON DELETE RESTRICT
    
            
);
"""



with engine.begin() as conn:
    # Create objects
    conn.execute(text(create_objects_sql))

    conn.commit()

# Write kindergartens data (replace if exists to keep it clean)
kg.to_sql(
    'kindergartens',
    engine,
    schema=schema,
    if_exists="replace",
    index=False
)



298